In [1]:
cur_dir = os.getcwd()
os.chdir(os.path.join(cur_dir, 'data'))
!git clone https://gitlab.cs.washington.edu/ALGES/TACL2015.git
!git clone https://github.com/chaochun/nlu-asdiv-dataset.git
!git clone https://github.com/openai/grade-school-math.git
os.chdir(cur_dir)

fatal: destination path 'TACL2015' already exists and is not an empty directory.
fatal: destination path 'nlu-asdiv-dataset' already exists and is not an empty directory.
fatal: destination path 'grade-school-math' already exists and is not an empty directory.


In [1]:
import os
from xml.etree import ElementTree
import numpy as np
import torch
import json
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
from io import StringIO
from contextlib import redirect_stdout
from termcolor import colored
import wandb

In [24]:
def set_all_seeds():
    """Set all seeds"""
    torch.manual_seed(0)
    np.random.seed(8)
    os.environ['PYTHONHASHSEED']=str(0)
    #torch.backends.cudnn.deterministic = True
    #torch.backends.cudnn.benchmark = False


In [2]:
import dataset_handler as dh
import helper_func as hf

gptj_model = "EleutherAI/gpt-j-6B"
codeparrot_model = "lvwerra/codeparrot"

In [3]:
"""Load gsm8k"""
priming_text_path = "data/priming_texts/gsm8k/gsm8k_fewer_alt_codegen.txt"
current_dataset = dh.init_dataset_from_name("gsm8k", primingtext_path = priming_text_path)

In [4]:
"""Load asdiv"""
priming_text_path = "data/priming_texts/asdiv/asdiv_prefix_codegen.txt"
current_dataset = dh.init_dataset_from_name("asdiv", primingtext_path = priming_text_path)

In [28]:
set_all_seeds()

sample_q_list, sample_a_list = current_dataset.sample_n_for_prompting(50)

print(colored(sample_q_list[0], "blue"))
print(colored(sample_a_list[0], "green"))

Write a program that prints the answer to the following question. When they got there, they saw that SFL is already filled with people. Every entrance has a long line, with 283 people waiting to get in. If SFL has 5 entrances, how many people in total are waiting to get in?
1415 (people)


In [6]:
"""CodeGen runs in the venv venv"""
model_args = hf.model_args()
model, tokenizer = hf.load_CodeGen(model_args)

loading parameters
loading parameters took 478.78s
loading tokenizer
loading tokenizer took 4.02s


In [ ]:
"""GPT-J and codeparrot models run in HFTest venv"""
tokenizer = AutoTokenizer.from_pretrained(gptj_model)
model = AutoModelForCausalLM.from_pretrained(gptj_model).half().eval().cuda()

In [23]:
gen_args = hf.codegen_gen_args()
gen_args.num_return_sequences = 5
gen_args.k = 3
gen_args.max_lenght_after_input = 250
gen_args.top_p = 1
gen_args.top_k = 200
gen_args.temperature = 0.7


set_all_seeds()
hf.testing_loop(current_dataset, tokenizer, model, sample_q_list, sample_a_list, gen_args)

TESTING STARTED
@sample 5 -> Pass@3 = 0.4
@sample 10 -> Pass@3 = 0.44000000000000006


Pass@3 = 0.44000000000000006


0.44000000000000006

In [34]:
config = hf.codegen_gen_args()
config.num_return_sequences = 5
config.k = 3
config.max_lenght_after_input = 250
config.top_p = 1
config.top_k = 50
config.temperature = 0.7

with wandb.init(project="PracticalWork", entity="antoniolopardo",config=config, name="@50-mid-top_k-codegen"):

        set_all_seeds()
        pass_at_k = hf.testing_loop(current_dataset, tokenizer, model, sample_q_list, sample_a_list, config)

        wandb.log({"pass_at_k": pass_at_k})

TESTING STARTED
@sample 5 -> Pass@3 = 0.4
@sample 10 -> Pass@3 = 0.44000000000000006
@sample 15 -> Pass@3 = 0.4571428571428572
@sample 20 -> Pass@3 = 0.4526315789473684
@sample 25 -> Pass@3 = 0.4791666666666667
@sample 30 -> Pass@3 = 0.496551724137931
@sample 35 -> Pass@3 = 0.47941176470588226
@sample 40 -> Pass@3 = 0.5179487179487179
@sample 45 -> Pass@3 = 0.5022727272727272
@sample 50 -> Pass@3 = 0.4755102040816327


Pass@3 = 0.4755102040816327



pass_at_k,▁
pass_at_k,0.47551
